In [19]:
import requests,time,datetime,json
from lxml import etree

# 职位信息
info_total  = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36',
    'Host': 'www.lagou.com',
    'Referer': 'https://www.lagou.com',
    'cookie': 'user_trace_token=20220507193801-3018e39a-6a91-42ac-85ab-6c67b25331f9; LGUID=20220507193801-c12c1d5a-98bb-4a5e-92b5-1e231261fec8; _ga=GA1.2.500245705.1651923481; LG_LOGIN_USER_ID=e05032402600528923cabd84dad825ab1a437552faf21bd8292dbd9b72299562; LG_HAS_LOGIN=1; showExpriedIndex=1; showExpriedCompanyHome=1; showExpriedMyPublish=1; hasDeliver=2; privacyPolicyPopup=false; index_location_city=%E5%85%A8%E5%9B%BD; RECOMMEND_TIP=true; gate_login_token=a235fd138eaea53e610ac0bb1d7557a3687f7c8cf5c351b54703f41b93895db2; JSESSIONID=ABAAAECAAEBABII8C8D61DCD20325FAE54F339F077EDF31; WEBTJ-ID=20220511123617-180b167476c2f4-08567b2d56ce37-17333273-1049088-180b167476d388; _putrc=F86169996E6139D9123F89F2B170EADC; login=true; unick=%E4%BB%A3%E4%B9%85; PRE_UTM=; PRE_HOST=; PRE_LAND=https%3A%2F%2Fwww.lagou.com%2F; LGSID=20220511123620-11d153ee-332a-486d-a3b5-c322db0a7e41; PRE_SITE=https%3A%2F%2Fwww.lagou.com; __SAFETY_CLOSE_TIME__18339134=1; _gat=1; _gid=GA1.2.934348774.1652243782; sensorsdata2015session=%7B%7D; TG-TRACK-CODE=index_navigation; __lg_stoken__=7046f6ede9a0d56c521ea5c4219789f5870b19799b18933a12a163c9a725d0b01f6ceb12866ac8f5a2122882c01b7db0b793176fca7224fb3152fc4e6b75f43751143ae06e29; X_HTTP_TOKEN=ae3aba4b26f08e7e4704422561430624b612c179ba; LGRID=20220511124114-d1982e06-57db-42f6-b612-e352ec3498c3; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2218339134%22%2C%22%24device_id%22%3A%221809e4ff5535f9-0f2e67896d90b5-17333273-1049088-1809e4ff5544d7%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E5%BC%95%E8%8D%90%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22http%3A%2F%2Flocalhost%3A8888%2F%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC%22%2C%22%24os%22%3A%22Windows%22%2C%22%24browser%22%3A%22Chrome%22%2C%22%24browser_version%22%3A%22101.0.4951.54%22%2C%22lagou_company_id%22%3A%22%22%7D%2C%22first_id%22%3A%221809e4ff5535f9-0f2e67896d90b5-17333273-1049088-1809e4ff5544d7%22%7D'
}

In [20]:
#jobs_info表
'''
job: 职位名
keyword: 关键词
place: 工作地点
requirement: 经验和学历要求
salary: 薪水
tags: 需要掌握的技术栈
welfare: 福利
pubtime: 职位发布时间
'''

def getJobInfo(keyword,page):
    url = f"https://www.lagou.com/wn/jobs?pn={page}&kd={keyword}&city=全国"
    r = requests.get(url,headers=headers)
    html = etree.HTML(r.text)
    jobs_list = html.xpath('//*[@id="jobList"]/div[1]')[0]
    for item in jobs_list:
        job = item.xpath('.//div[@class="position__21iOS"]//a/text()')
        name = job[0]
        place = job[1][1:-1].split("·")[0]
        salary = item.xpath('.//span[@class="money__3Lkgq"]/text()')[0]
        requirement = ','.join(item.xpath('.//div[@class="p-bom__JlNur"]/text()')[0].split(" / "))
        tags = ','.join(item.xpath('.//div[@class="ir___QwEG"]//text()'))
        welfare = item.xpath('.//div[@class="item-bom__cTJhu"]/div/text()')[0][1:-1]
        pubtime = item.xpath('.//div[@class="p-top__1F7CL"]/span/text()')[0]
        if '天前发布' in pubtime:
            i = 0-int(pubtime.replace("天前发布",""))
            day = datetime.datetime.now()+datetime.timedelta(days=i)
            pubtime = str(day.strftime('%Y-%m-%d'))
        else:
            day = datetime.datetime.now()
            pubtime = str(day.strftime('%Y-%m-%d'))
        info = {
            'job': name,
            'keyword': keyword,
            'place': place,    
            'requirement': requirement,
            'salary':salary,
            'tags': tags,
            'welfare': welfare,
            'pubtime': pubtime
        }
        info_total.append(info)
def main():
    for keyword in ["Java","大数据","Python","Android","C++","Go","产品经理","网页设计师","HR","游戏开发","小程序开发"]:
        for page in range(1,6):
            url = f"https://www.lagou.com/wn/jobs?pn={page}&kd={keyword}&city=全国"
            getJobInfo(keyword,page)
        print(keyword+"爬取成功")
        time.sleep(1)
def saveToJson():
    f = open("res/jobsInfo.json",'w',encoding='utf-8')
    f.write(json.dumps(info_total))
    f.close()
    print("保存成功")
if __name__ == '__main__':
    main()
    saveToJson()

Java爬取成功
大数据爬取成功
Python爬取成功
Android爬取成功
C++爬取成功
Go爬取成功
产品经理爬取成功
网页设计师爬取成功
HR爬取成功
游戏开发爬取成功
小程序开发爬取成功
保存成功


In [21]:
print(info_total)

[{'job': '初级java开发工程师', 'keyword': 'Java', 'place': '深圳', 'requirement': '经验不限,本科', 'salary': '6k-12k', 'tags': 'Linux,HTML', 'welfare': '五险一金、绩效奖金、股权激励等', 'pubtime': '2022-05-11'}, {'job': 'java工程师', 'keyword': 'Java', 'place': '西安', 'requirement': '经验3-5年,本科', 'salary': '10k-15k', 'tags': '医疗｜保健｜美容,IT技术服务｜咨询', 'welfare': '全额社保公积金', 'pubtime': '2022-05-11'}, {'job': 'java开发工程师', 'keyword': 'Java', 'place': '成都', 'requirement': '经验不限,本科', 'salary': '4k-8k', 'tags': '软件服务｜咨询,在线医疗,Spring,MyBatis,Vue', 'welfare': '前后端分离', 'pubtime': '2022-05-11'}, {'job': 'java开发工程师', 'keyword': 'Java', 'place': '杭州', 'requirement': '经验不限,本科', 'salary': '4k-8k', 'tags': '电子政务,互联网', 'welfare': '作为团队核心技术骨干人员、项目经理培养', 'pubtime': '2022-05-11'}, {'job': 'java开发工程师', 'keyword': 'Java', 'place': '苏州', 'requirement': '经验不限,本科', 'salary': '4k-8k', 'tags': '软件服务｜咨询,制造业,Web后端开发,CSS,MySQL', 'welfare': '培训，职业晋升，企业文化', 'pubtime': '2022-05-11'}, {'job': 'java开发工程师', 'keyword': 'Java', 'place': '成都', 'requirement': '经验不限

In [2]:
import sys
from hdfs.client import Client
from hdfs.client import InsecureClient

In [4]:
client = InsecureClient("http://192.168.88.138:50070", user='hadoop');

# 上传文件到hdfs
def put_to_hdfs(client, local_path, hdfs_path):
    client.upload(hdfs_path, local_path, cleanup=True)
    
# move_or_rename(client,'/input/2.csv', '/input/emp.csv')
# read_hdfs_file(client,'/input/emp.csv')
put_to_hdfs(client, './zl0507.csv', '/task2-Crawler')